In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import yfinance as yf
import requests
from bs4 import BeautifulSoup

In [ ]:
url_stock_list_goodinfo = "https://goodinfo.tw/tw/Lib.js/StockTW_ID_NM_List.js"
res = requests.get(url_stock_list_goodinfo)
soup = BeautifulSoup(res.text, 'html.parser')

list_ = soup.string.split("','")
list_4 = [i.split(" ")[0] for i in list_[2:] if len(i.split(" ")[0]) == 4][:2179]
ticker_dict = {}
for ticker in list_4:
    df = yf.download(f"{ticker}.TW", start='2024-07-10', progress=False)
    if len(df) != 0:
        ticker_dict[ticker] = "TW"
    else:
        df = yf.download(f"{ticker}.TWO", start='2024-07-12', progress=False)
        if len(df) != 0:
            ticker_dict[ticker] = "TWO"
        else:
            ticker_dict[ticker] = "0"

import json
with open("../db/tw/symbol/goodinfo_tw.json", "w") as f:
    json.dump({i:value for i, value in ticker_dict.items() if value == "TW"}, f, indent=4)

with open("../db/tw/symbol/goodinfo_two.json", "w") as f:
    json.dump({i:value for i, value in ticker_dict.items() if value == "TWO"}, f, indent=4)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import os
import json
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
os.chdir("../db/")

In [ ]:
arrs = []
try_ = 3
sleep_ = 3
season_dict = {
1: '03-31',
2: "06-30",
3: "09-30",
4: "12-31",
}
slices = {
    'df_current': slice(4, 13),
    'df_noncurrent': slice(14, 19),
    'df_currentlia': slice(20, 30),
    'df_noncurrentlia': slice(31, 38),
    'df_total_lia': -1,
    'df_equity': -3
}
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model, useful for running on servers
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems

# Set up the Chrome WebDriver with the specified options
driver = webdriver.Chrome(options=chrome_options)
url = 'https://mops.twse.com.tw/mops/web/t164sb04'
driver.get(url)
wait = WebDriverWait(driver, 20)
time.sleep(2)
select_element_isnew = wait.until(EC.presence_of_element_located((By.ID, 'isnew')))
select_isnew = Select(select_element_isnew)
company_code_input = wait.until(EC.presence_of_element_located((By.ID, 'co_id')))
year_input = driver.find_element(By.ID, 'year')
select_element_Season = wait.until(EC.presence_of_element_located((By.ID, 'season')))
select_season = Select(select_element_Season)
search_button = driver.find_element(By.CSS_SELECTOR, 'input[type="button"][value=" 查詢 "]')
for year in [109, 107]:
    if year != 109:
        year_input.clear()
        list_season = [1, 2, 3, 4]
    else:
        list_season = [1]
    for season in list_season:
        for ticker in tqdm(list_tw_stock, desc=f"{year}, {season}"):
            wait = WebDriverWait(driver, 20)
            select_isnew.select_by_visible_text('歷史資料')

            company_code_input.clear()
            company_code_input.send_keys(ticker)

            year_input.send_keys(str(year))

            select_season.select_by_visible_text(str(season))
            search_button.click()

            wait.until(EC.presence_of_element_located((By.ID, 'table01')))
            time.sleep(2)
            data = []
            df = None
            for i in range(try_):
                try:
                    table = driver.find_element(By.CLASS_NAME, 'hasBorder')
                    rows = table.find_elements(By.TAG_NAME, "tr")
                    for row in rows:
                        cols = row.find_elements(By.TAG_NAME, 'td')
                        cols = [col.text for col in cols]
                        data.append(cols)
                        df = pd.DataFrame(data)
                    break
                except Exception as e:
                    # financial statements not exist
                    break
            if df is None:
                continue
            try:
                df_sp = df.iloc[:, [0, 1, 3]]
                dfs = {name: df_sp.iloc[s] if isinstance(s, slice) else df_sp.iloc[[s], :] for name, s in slices.items()}
                df_concat = pd.concat(dfs.values())
                df_concat.insert(2, "season_da1", f"{1911+year}-{season_dict[season]}")
                df_concat.insert(3, "ticker", ticker)
                df_concat.insert(4, "col_name", df_concat.iloc[:, 0])
                df_concat.insert(6, "season_da2", f"{1911+year-1}-{season_dict[season]}")
                df_concat.insert(7, "ticker1", ticker)
                array_f = df_concat.values.reshape(int(df_concat.shape[0]*2), int(df_concat.shape[1]/2))
                arrs.append(array_f)
            except IndexError:
                continue
    df_long_format = pd.DataFrame(sum([array_f.tolist() for array_f in arrs], []))
    df_long_format.columns = ['colname', 'value', ';season_end', "ticker"]
    df_long_format.to_parquet(f'../db/tw/financials/CI_{year}.parquet')